<a href="https://colab.research.google.com/github/LC1332/Prophet-Andrew-Ng/blob/main/langchain/%E6%9D%8E%E9%B2%81%E9%B2%81%E5%AD%A6LangChain_21_%E5%A4%9A%E6%96%87%E6%A1%A3%E7%9A%84%E6%A3%80%E7%B4%A2%E5%92%8CQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 李鲁鲁学LangChain 21 多文档的检索和QA

这是李鲁鲁的学习笔记，对Sam Witteveen的油管课程的记录

增加了李鲁鲁学习时候的吐槽

因为一个独立的md文件导入知乎会更方便，所以这个课程的笔记会记录在md中

比如这节课在

https://github.com/LC1332/Prophet-Andrew-Ng/blob/main/langchain/%E6%9D%8E%E9%B2%81%E9%B2%81%E5%AD%A6LangChain%2021.md

欢迎访问[骆驼项目](https://github.com/LC1332/Luotuo-Chinese-LLM)主页和点赞


In [1]:
!pip -q install langchain openai tiktoken chromadb sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip show langchain

Name: langchain
Version: 0.0.240
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [3]:
!wget -q https://github.com/LC1332/Prophet-Andrew-Ng/raw/main/langchain/%E5%93%88%E5%88%A9%E6%B3%A2%E7%89%B9%E7%94%B5%E5%AD%90%E7%89%88.zip

In [4]:
!unzip -q 哈利波特电子版.zip -d harry

In [ ]:
# !wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip
# !unzip -q new_articles.zip -d new_articles

# LangChain multi-doc retriever with ChromaDB

***New Points***
- Multiple Files
- ChromaDB
- Source info
- gpt-3.5-turbo API

## Setting up LangChain


In [5]:
import os

os.environ["OPENAI_API_KEY"] = "sk-0hE0"

In [6]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader


## Load multiple and process documents

In [7]:
!ls /content/harry/Book/CN/book_4

chapter_10.txt	chapter_18.txt	chapter_25.txt	chapter_32.txt	chapter_6.txt
chapter_11.txt	chapter_19.txt	chapter_26.txt	chapter_33.txt	chapter_7.txt
chapter_12.txt	chapter_1.txt	chapter_27.txt	chapter_34.txt	chapter_8.txt
chapter_13.txt	chapter_20.txt	chapter_28.txt	chapter_35.txt	chapter_9.txt
chapter_14.txt	chapter_21.txt	chapter_29.txt	chapter_36.txt
chapter_15.txt	chapter_22.txt	chapter_2.txt	chapter_3.txt
chapter_16.txt	chapter_23.txt	chapter_30.txt	chapter_4.txt
chapter_17.txt	chapter_24.txt	chapter_31.txt	chapter_5.txt


In [8]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('/content/harry/Book/CN/book_4', glob="./*.txt", loader_cls=TextLoader)

documents = loader.load()

In [33]:
# from langchain.text_splitter import CharacterTextSplitter

# text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
#     chunk_size=100, chunk_overlap=10
# )
# texts = text_splitter.split_documents(documents)

In [41]:
print(len(documents))

36


In [12]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=40)
texts = text_splitter.split_documents(documents)

In [13]:
len(texts)

878

In [14]:
# print(len(texts[3]))
texts[3]


Document(page_content='“晚上好，女士们，先生们，鬼魂们，还有——特别是——贵宾们，”邓布利多说，笑眯眯地望着那些外国学生，“我怀着极大的喜悦，欢迎你们来到霍格沃茨。我希望并且相信，你们在这里会感到舒适愉快的。”\n一个布斯巴顿的女生仍然用围巾紧紧裹着脑袋，发出一声无疑是讥讽的冷笑。\n“又没有人强迫你们留下来！”赫敏小声说，她被那个女生惹恼了。\n“争霸赛将于宴会结束时正式开始。”邓布利多说，“我现在邀请大家尽情地吃喝，就像在自己家里一样！”\n他坐下了，哈利看见卡卡洛夫立刻靠上前去，跟他交谈。\n他们面前的盘子里又像往常一样堆满了食物。厨房里的那些家养小精灵似乎使出了浑身解数。哈利还从没见过这么丰盛的菜肴，五花八门地摆在他们面前，其中有几样肯定是外国风味。\n“那是什么？”罗恩问，指着大块牛排腰子布丁旁边的一大盘东西，看样子像是海鲜大杂烩。\n“法式杂鱼汤。”赫敏说。\n“好家伙。”罗恩说。\n“这是法国菜，”赫敏说，“我前年暑假的时候吃过。味道很鲜美的。”\n“我就相信你吧。”罗恩说着，给自己盛了一些黑布丁。', metadata={'source': '/content/harry/Book/CN/book_4/chapter_15.txt'})

## OpenAI Embedding


In [25]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [16]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [26]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

In [18]:
!pip install -q InstructorEmbedding

In [19]:

from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="silk-road/luotuo-bert-medium",
                                                      model_kwargs={"device": "cuda"})


/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


Some weights of BertModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/silk-road_luotuo-bert-medium and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db_local'

## Here is the nmew embeddings being used
embedding_local = instructor_embeddings

vectordb_local = Chroma.from_documents(documents=texts,
                                 embedding=embedding_local,
                                 persist_directory=persist_directory)

In [21]:
# persiste the db to disk
vectordb_local.persist()
vectordb_local = None

In [28]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb_local = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding_local)

## create the DB

In [29]:
retriever = vectordb.as_retriever(search_kwargs={"k": 6})
retriever_local = vectordb_local.as_retriever(search_kwargs={"k": 6})



In [30]:
docs = retriever.get_relevant_documents("火焰杯比赛是什么?")
docs_local = retriever_local.get_relevant_documents("火焰杯比赛是什么?")


In [ ]:
# # Embed and store the texts
# # Supplying a persist_directory will store the embeddings on disk
# persist_directory = 'db'

# ## here we are using OpenAI embeddings but in future we will swap out to local embeddings
# embedding = OpenAIEmbeddings()

# vectordb = Chroma.from_documents(documents=texts[:10],
#                                  embedding=embedding,
#                                  persist_directory=persist_directory)

In [20]:
print(vectordb)

In [ ]:
# # persiste the db to disk
# vectordb.persist()
# vectordb = None

In [ ]:
# # Now we can load the persisted database from disk, and use it as normal.
# vectordb = Chroma(persist_directory=persist_directory,
#                   embedding_function=embedding)

## Make a retriever

In [52]:
retriever = vectordb.as_retriever()

In [53]:
docs = retriever.get_relevant_documents("火焰杯比赛是什么?")

In [54]:
len(docs)

4

In [31]:
print(docs[1])

page_content='她非常生气地瞪了斯内普教授一眼。\n“克劳奇先生……巴格曼先生，”卡卡洛夫说，声音又变得油滑起来，“你们二位是我们的——嗯——客观的裁判。你们肯定也认为这件事是极不合适的，是吗？”\n巴格曼用手帕擦了擦他圆乎乎的娃娃脸，转眼望着克劳奇先生。克劳奇先生站在炉火的光圈外面，他的脸一半隐藏在阴影中。他显得有点儿怪异，那半边黑影使他显得苍老了许多，看上去简直有点儿像个骷髅。不过当他说话时，声音还和往常一样生硬。\n“我们必须遵守章程，章程里明确规定，凡是名字从火焰杯里喷出来的人，都必须参加三强争霸赛的竞争。”\n“嘿，巴蒂把章程背得滚瓜烂熟。”巴格曼说，脸上绽开笑容，回过头来望着卡卡洛夫和马克西姆夫人，似乎事情已经圆满解决了。\n“我坚持要我的其他学生重新报名。”卡卡洛夫说。他的声音不再圆滑，笑容也消失了，脸上的表情难看极了，“你们必须把火焰杯重新摆出来，我们要不断地往里面加进名字，直到每个学校产生两位勇士。这样才算公平，邓布利多。”\n“可是卡卡洛夫，这恐怕不成，”巴格曼说，“火焰杯刚刚熄灭——要到下届争霸赛时才会重新燃起——”' metadata={'source': '/content/harry/Book/CN/book_4/chapter_16.txt'}


In [75]:
retriever = vectordb.as_retriever(search_kwargs={"k": 6})

In [57]:
retriever.search_type

'similarity'

In [58]:
retriever.search_kwargs

{'k': 4}

## Make a chain

In [32]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo')

In [33]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name='gpt-3.5-turbo'),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [34]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [35]:
# full example
query = "火焰杯竞赛是什么?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

火焰杯竞赛是一个友谊竞争，由三所欧洲最大的魔法学校参与。每五年举行一次，轮流由霍格沃茨、布斯巴顿和德姆斯特朗三所学校主办。每个学校会选出一名勇士代表他们参加比赛。在比赛中，勇士们会进行三种不同的魔法项目的竞争。这个竞赛旨在促进不同国家之间年轻巫师们的友谊，但由于过去发生了许多死亡事件，比赛曾经中断过一段时间。


Sources:
/content/harry/Book/CN/book_4/chapter_15.txt
/content/harry/Book/CN/book_4/chapter_15.txt
/content/harry/Book/CN/book_4/chapter_16.txt
/content/harry/Book/CN/book_4/chapter_15.txt
/content/harry/Book/CN/book_4/chapter_15.txt
/content/harry/Book/CN/book_4/chapter_11.txt


火焰杯竞赛是《哈利·波特与火焰杯》中的一个魔法竞赛活动。它是三个魔法学院（霍格沃茨、德姆斯特朗、博巴顿）之间的一项比赛，每个学院都派出一名代表参加。参赛者从火焰杯中被选中，并要面对一系列的考验，包括战胜火龙、穿越迷宫等等。最终，获得最高分的参赛者将成为火焰杯的胜利者。


In [36]:
# full example
query = "为什么说哈利波特能报名参加火焰杯是一个反常现象?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

哈利波特报名参加火焰杯被认为是一个反常现象，因为根据比赛规则，只有17岁及以上的学生才有资格参加，而哈利只有14岁。此外，火焰杯投票的过程应该是匿名的，没有人应该知道是谁将哈利的名字投进去的。因此，哈利的参赛被认为是异常和不正常的。


Sources:
/content/harry/Book/CN/book_4/chapter_16.txt
/content/harry/Book/CN/book_4/chapter_17.txt
/content/harry/Book/CN/book_4/chapter_16.txt
/content/harry/Book/CN/book_4/chapter_21.txt
/content/harry/Book/CN/book_4/chapter_11.txt
/content/harry/Book/CN/book_4/chapter_9.txt


In [69]:
# full example
query = "为什么说哈利波特能报名参加火焰杯是一个反常现象?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

根据给出的文本，没有提到哈利报名参加火焰杯是一个反常现象。所以不能确定为什么会这样说。


Sources:
/content/harry/Book/CN/book_4/chapter_19.txt
/content/harry/Book/CN/book_4/chapter_16.txt
/content/harry/Book/CN/book_4/chapter_31.txt
/content/harry/Book/CN/book_4/chapter_6.txt
/content/harry/Book/CN/book_4/chapter_30.txt
/content/harry/Book/CN/book_4/chapter_27.txt
/content/harry/Book/CN/book_4/chapter_26.txt
/content/harry/Book/CN/book_4/chapter_4.txt


根据提供的文本，没有明确说明为什么哈利波特能报名参加火焰杯是一个反常现象。然而，根据哈利和罗恩的对话，罗恩表达了对哈利报名参加火焰杯的怀疑，并认为哈利可能在昨天夜里偷偷报名。这可能是因为火焰杯只允许17岁以上的魔法师参加，而哈利只有14岁。因此，哈利能够报名参加火焰杯可能被认为是不寻常的。

In [37]:
# full example
query = "让哈利波特参加这一次的火焰杯是一个阴谋吗?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

根据这些片段的描述，我们不能确定是否有一个明确的阴谋或者谁是背后的幕后黑手。哈利自己也在怀疑是否有人故意让他参加比赛，但他并没有得出确凿的结论。我们只能推测可能有人试图捉弄他或者让他出洋相，但具体动机和背后的计划仍然不清楚。


Sources:
/content/harry/Book/CN/book_4/chapter_16.txt
/content/harry/Book/CN/book_4/chapter_9.txt
/content/harry/Book/CN/book_4/chapter_17.txt
/content/harry/Book/CN/book_4/chapter_15.txt
/content/harry/Book/CN/book_4/chapter_16.txt
/content/harry/Book/CN/book_4/chapter_30.txt


In [65]:
# full example
query = "让哈利波特参加这一次的火焰杯是一个阴谋吗?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

根据提供的文本，没有直接的证据表明让哈利参加火焰杯是一个阴谋。然而，罗恩对哈利的怀疑和丽塔·斯基特的追问可能暗示着一些人对哈利参加火焰杯持怀疑态度。然而，我们不能确定是否存在一个明确的阴谋。


Sources:
/content/harry/Book/CN/book_4/chapter_16.txt
/content/harry/Book/CN/book_4/chapter_19.txt
/content/harry/Book/CN/book_4/chapter_31.txt
/content/harry/Book/CN/book_4/chapter_6.txt


根据提供的文本，没有直接的证据表明让哈利参加火焰杯是一个阴谋。然而，罗恩对哈利的怀疑和丽塔·斯基特的追问可能暗示着一些人对哈利参加火焰杯持怀疑态度。然而，我们不能确定是否存在一个明确的阴谋。


In [38]:
# full example
query = "让哈利波特参加火焰杯和伏地魔有什么关系？"
llm_response = qa_chain(query)
process_llm_response(llm_response)

根据给定的文本，没有明确提到为什么哈利波特被选中参加火焰杯和伏地魔之间的关系。然而，可以推测出以下可能的关系：

1. 伏地魔可能是通过他的食死徒的帮助，将哈利的名字放入火焰杯，并确保他在比赛中获胜。这可能是为了吸引哈利到伏地魔的身边，以便他能够控制哈利或对他进行某种威胁。

2. 另一种可能是，伏地魔可能希望哈利参加比赛，以便能够对他加以折磨或试图杀死他。这种情况下，伏地魔可能认为比赛中的危险项目会给哈利带来极大的伤害或甚至致命。

需要注意的是，这些只是猜测，没有明确的证据支持。


Sources:
/content/harry/Book/CN/book_4/chapter_18.txt
/content/harry/Book/CN/book_4/chapter_32.txt
/content/harry/Book/CN/book_4/chapter_15.txt
/content/harry/Book/CN/book_4/chapter_21.txt
/content/harry/Book/CN/book_4/chapter_16.txt
/content/harry/Book/CN/book_4/chapter_36.txt


根据提供的文本，哈利波特参加火焰杯和伏地魔之间没有直接的关系。哈利波特参加火焰杯是因为他的名字被放入了高脚杯，导致他被选为霍格沃茨学校的第四个勇士。而伏地魔的复活是在哈利参加火焰杯之后发生的，这两个事件是在不同的时间和背景下发生的。


In [39]:
# full example
query = "哈利波特打得过龙吗?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

根据给出的文本，哈利波特在三强争霸赛中的第二个项目中面对的是一条龙。尽管他感到紧张和不确定，但最终哈利使用了鳃囊草并成功地完成了任务，夺回了被龙抢走的东西。因此，可以说哈利波特在一定程度上打败了龙。


Sources:
/content/harry/Book/CN/book_4/chapter_17.txt
/content/harry/Book/CN/book_4/chapter_19.txt
/content/harry/Book/CN/book_4/chapter_25.txt
/content/harry/Book/CN/book_4/chapter_14.txt
/content/harry/Book/CN/book_4/chapter_25.txt
/content/harry/Book/CN/book_4/chapter_19.txt


根据给出的文本，哈利波特在三强争霸赛中的第二个项目中面对的是一条龙。尽管他感到紧张和不确定，但最终哈利使用了鳃囊草并成功地完成了任务，夺回了被龙抢走的东西。因此，可以说哈利波特在一定程度上打败了龙。



In [40]:
# full example
query = "哈利波特如何在火焰杯的第一场比赛中获胜呢?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

在火焰杯的第一场比赛中，哈利波特使用了鳃囊草，这是一种能够让人在水下呼吸的植物。他用鳃囊草在湖底找到了人质，确保了所有人质的安全返回。虽然他超过了规定时间，但人鱼女首领告诉裁判，哈利是第一个找到人质的，并且他的担心是确保所有人的安全，而不仅仅是他自己的人质。裁判们认为这体现了高尚的道德风范，所以哈利得到了一定的分数，并与塞德里克打平。


Sources:
/content/harry/Book/CN/book_4/chapter_7.txt
/content/harry/Book/CN/book_4/chapter_30.txt
/content/harry/Book/CN/book_4/chapter_7.txt
/content/harry/Book/CN/book_4/chapter_15.txt
/content/harry/Book/CN/book_4/chapter_25.txt
/content/harry/Book/CN/book_4/chapter_19.txt


In [41]:
# full example
query = "有办法在水下憋气很久吗？"
llm_response = qa_chain(query)
process_llm_response(llm_response)

在文中的描述中，并没有提到有关在水下憋气的技巧或方法。因此，我无法回答您的问题。


Sources:
/content/harry/Book/CN/book_4/chapter_25.txt
/content/harry/Book/CN/book_4/chapter_25.txt
/content/harry/Book/CN/book_4/chapter_25.txt
/content/harry/Book/CN/book_4/chapter_25.txt
/content/harry/Book/CN/book_4/chapter_24.txt
/content/harry/Book/CN/book_4/chapter_11.txt


In [42]:
# full example
query = "有办法让哈利在水下活动很久吗？"
llm_response = qa_chain(query)
process_llm_response(llm_response)

根据给出的上下文，尽管哈利和他的朋友们在寻找可以使哈利在水下存活的办法，但他们没有找到任何有效的方法。因此，没有明确的办法可以让哈利在水下活动很久。


Sources:
/content/harry/Book/CN/book_4/chapter_25.txt
/content/harry/Book/CN/book_4/chapter_25.txt
/content/harry/Book/CN/book_4/chapter_24.txt
/content/harry/Book/CN/book_4/chapter_25.txt
/content/harry/Book/CN/book_4/chapter_25.txt
/content/harry/Book/CN/book_4/chapter_25.txt


In [43]:
# full example
query = "鳃囊草在小说中的作用是什么？"
llm_response = qa_chain(query)
process_llm_response(llm_response)

在小说中，鳃囊草的作用是让哈利在第二个任务中呼吸水下并且生存。哈利需要在一个湖里找到并夺回被抢走的东西，而鳃囊草可以让他在水下呼吸和移动。


Sources:
/content/harry/Book/CN/book_4/chapter_34.txt
/content/harry/Book/CN/book_4/chapter_25.txt
/content/harry/Book/CN/book_4/chapter_32.txt
/content/harry/Book/CN/book_4/chapter_17.txt
/content/harry/Book/CN/book_4/chapter_17.txt
/content/harry/Book/CN/book_4/chapter_4.txt


In [44]:
# full example
query = "伏地魔在小说中干了什么?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

在小说中，伏地魔干了许多事情。他是主要的反派角色，他试图统治魔法世界并消灭所有不纯血统的巫师和麻瓜（非魔法人士）。他建立了食死徒组织，招募追随者，并在整个系列中进行了多次攻击和战斗。他试图杀死哈利·波特，因为据他的预言，只有哈利可以摧毁他。伏地魔还试图复活并获得永生，为此他追求魔法石、霍格沃茨的三强争霸赛和其他力量和物品。他通过附身和其他黑暗魔法手段来延长自己的生命，并试图掌握死亡圣器以获得不朽。总的来说，伏地魔在整个系列中是一个极具威胁和邪恶的角色。


Sources:
/content/harry/Book/CN/book_4/chapter_33.txt
/content/harry/Book/CN/book_4/chapter_32.txt
/content/harry/Book/CN/book_4/chapter_36.txt
/content/harry/Book/CN/book_4/chapter_32.txt
/content/harry/Book/CN/book_4/chapter_32.txt
/content/harry/Book/CN/book_4/chapter_35.txt


在小说中，伏地魔干了许多事情。他是主要的反派角色，他试图统治魔法世界并消灭所有不纯血统的巫师和麻瓜（非魔法人士）。他建立了食死徒组织，招募追随者，并在整个系列中进行了多次攻击和战斗。他试图杀死哈利·波特，因为据他的预言，只有哈利可以摧毁他。伏地魔还试图复活并获得永生，为此他追求魔法石、霍格沃茨的三强争霸赛和其他力量和物品。他通过附身和其他黑暗魔法手段来延长自己的生命，并试图掌握死亡圣器以获得不朽。总的来说，伏地魔在整个系列中是一个极具威胁和邪恶的角色。


In [45]:
# full example
query = "为什么伏地魔的阿瓦达索命没有干掉哈利波特？"
llm_response = qa_chain(query)
process_llm_response(llm_response)

根据提供的文本，当哈利和伏地魔的魔杖相遇时，两根魔杖之间产生了一个金色的光束，将他们困在了一起。这导致伏地魔的阿瓦达索命咒无法直接对哈利造成伤害。虽然具体的原因没有明确提及，但可以推测这是由于哈利的魔杖和伏地魔的魔杖之间存在一种特殊的连接，使得哈利免受伏地魔的攻击。


Sources:
/content/harry/Book/CN/book_4/chapter_33.txt
/content/harry/Book/CN/book_4/chapter_34.txt
/content/harry/Book/CN/book_4/chapter_29.txt
/content/harry/Book/CN/book_4/chapter_36.txt
/content/harry/Book/CN/book_4/chapter_18.txt
/content/harry/Book/CN/book_4/chapter_32.txt


根据提供的文本，当哈利和伏地魔的魔杖相遇时，两根魔杖之间产生了一个金色的光束，将他们困在了一起。这导致伏地魔的阿瓦达索命咒无法直接对哈利造成伤害。虽然具体的原因没有明确提及，但可以推测这是由于哈利的魔杖和伏地魔的魔杖之间存在一种特殊的连接，使得哈利免受伏地魔的攻击。


## 用英文试试看

我对这些结果不是很满意

让我们使用英文小说来实验看看

In [81]:

from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})


load INSTRUCTOR_Transformer
max_seq_length  512


In [82]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('/content/harry/Book/EN/book_4', glob="./*.txt", loader_cls=TextLoader)

documents = loader.load()

In [83]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
retriever = vectordb.as_retriever()



In [ ]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [ ]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

一样建立英文retriever

In [ ]:
# full example
query = "How much money did Pando raise?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Pando raised $30 million in a Series B round, bringing its total raised to $45 million.


Sources:
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt


In [ ]:
# break it down
query = "What is the news about Pando?"
llm_response = qa_chain(query)
# process_llm_response(llm_response)
llm_response

{'query': 'What is the news about Pando?',
 'result': ' Pando raised $30 million in a Series B round, bringing their total raised to $45 million. The money will be used to expand their global sales, marketing and delivery capabilities, as well as to hire more staff and explore strategic partnerships and acquisitions.',
 'source_documents': [Document(page_content='Pando was co-launched by Jayakrishnan and Abhijeet Manohar, who previously worked together at iDelivery, an India-based freight tech marketplace — and their first startup. The two saw firsthand manufacturers, distributors and retailers were struggling with legacy tech and point solutions to understand, optimize and manage their global logistics operations — or at least, that’s the story Jayakrishnan tells.\n\n“Supply chain leaders were trying to build their own tech and throwing people at the problem,” he said. “This caught our attention — we spent months talking to and building for enterprise users at warehouses, factories, f

In [ ]:
query = "Who led the round in Pando?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Iron Pillar and Uncorrelated Ventures.


Sources:
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt


In [ ]:
query = "What did databricks acquire?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Databricks acquired Okera, a data governance platform with a focus on AI.


Sources:
new_articles/05-03-databricks-acquires-ai-centric-data-governance-platform-okera.txt
new_articles/05-03-databricks-acquires-ai-centric-data-governance-platform-okera.txt


In [ ]:
query = "What is generative ai?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Generative AI is a type of artificial intelligence that is used to create new content associated with a company, such as content for a website or ads. It can also be used to automate processes and workflows.


Sources:
new_articles/05-04-slack-updates-aim-to-put-ai-at-the-center-of-the-user-experience.txt
new_articles/05-03-nova-is-building-guardrails-for-generative-ai-content-to-protect-brand-integrity.txt


In [ ]:
query = "Who is CMA?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The CMA stands for the Competition and Markets Authority.


Sources:
new_articles/05-04-cma-generative-ai-review.txt
new_articles/05-04-cma-generative-ai-review.txt


In [ ]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7f9f7dc82aa0>)

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


## Deleteing the DB

In [ ]:
!zip -r db.zip ./db

  adding: db/ (stored 0%)
  adding: db/chroma-collections.parquet (deflated 50%)
  adding: db/index/ (stored 0%)
  adding: db/index/index_metadata_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 5%)
  adding: db/index/uuid_to_id_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 39%)
  adding: db/index/index_59c51927-205d-4fd7-88d8-c7ba851bd2a5.bin (deflated 17%)
  adding: db/index/id_to_uuid_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl (deflated 35%)
  adding: db/chroma-embeddings.parquet (deflated 29%)


In [ ]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# delete the directory
!rm -rf db/

## Starting again loading the db

restart the runtime

In [ ]:
!unzip db.zip

Archive:  db.zip
   creating: db/
  inflating: db/chroma-collections.parquet  
   creating: db/index/
  inflating: db/index/index_metadata_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/index/uuid_to_id_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/index/index_59c51927-205d-4fd7-88d8-c7ba851bd2a5.bin  
  inflating: db/index/id_to_uuid_59c51927-205d-4fd7-88d8-c7ba851bd2a5.pkl  
  inflating: db/chroma-embeddings.parquet  


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [ ]:
persist_directory = 'db'
embedding = OpenAIEmbeddings()

vectordb2 = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding,
                   )

retriever = vectordb2.as_retriever(search_kwargs={"k": 2})

In [ ]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "How much money did Pando raise?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Pando raised $30 million in a Series B round, bringing its total raised to $45 million.


Sources:
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt
new_articles/05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt


### Chat prompts

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}
